In [1]:
using Revise
import SSD

Precompiling SSD
  ✓ SSD
  1 dependency successfully precompiled in 65 seconds. 71 already precompiled.


In [2]:
using Clustering
using CPUTime
using CSV
using DataFrames
using Dates
using DelimitedFiles
using Distributions
using Distances
using Gurobi
using JuMP
using LinearAlgebra
using Plots
using Random

In [ ]:
using TimerOutputs

# Create a TimerOutput, this is the main type that keeps track of everything.
const to = TimerOutput()

In [ ]:
function get_res(file)
    extract = false
    instance = ""
    method = ""
    lb = -99
    ub = -99
    cpu_t = -99
    status = ""
    iter = -99
    mess = ""
    backup = ""
    i=1
    open(file,"r") do f  
        while !eof(f)
            line = readline(f)
            sline = split(line)
            if i == 7
                backup = line
            end
            if size(sline)[1] > 0
                if extract==true
                    if sline[1] == "Instance:"
                        instance = split(split(sline[2], "/")[end],".")[1]
                    elseif sline[1] == "Method:"
                        method = sline[2]
                    elseif sline[1] == "LB:"
                        lb = parse(Float64,sline[2])
                    elseif sline[1] == "UB:"
                        ub = parse(Float64,sline[2])
                    elseif sline[1] == "CPU_t:"
                        cpu_t = parse(Float64,sline[2])
                    elseif sline[1] == "Status:"
                        status = sline[2]
                    elseif sline[1] == "Iter:"
                        iter = parse(Float64,sline[2])
                    end
                end
                if sline[1] == "RESULTS"                
                    extract = true
                end
                if sline[1] == "slurmstepd:"
                    println(file)
                    mess = join(sline[end-2:end-1]," ")
                    if backup != ""
                        bk = split(backup,":")
                        instance = split(bk[2],",")[1]
                        method=bk[end]
                        println(bk)
                        println(instance)
                        println(method)
                    end
                    return instance, method, lb, ub, cpu_t, status, iter, mess
                end
            end
            i+=1
        end
    end
    return instance, method, lb, ub, cpu_t, status, iter, mess
end

In [ ]:
df_res1 = DataFrame(inst=String[], method=String[], lb=Float64[], ub=Float64[], cpu_t=Float64[],status=String[],iter=Float64[], error=String[])
directory = "results/"
files = readdir(directory)
for filename in files
# filename = "ssd_1218114_1.out"
    # println(filename)
    instance, method, lb, ub, cpu_t, status, iter, mess = get_res(directory*filename)
    push!(df_res1, (instance, method, lb, ub, cpu_t, status, iter, mess));
end

In [ ]:
sort!(df_res1,[:inst])

In [ ]:
CSV.write("results/results.csv",df_res1)

In [ ]:
function extract_pmed(file)
    open(file,"r") do f        
        line = readline(f)
        sline = split(line)
        nJ = parse(Int64,sline[2])
        nI = parse(Int64,sline[1]) - nJ
        Icoords = Matrix{Float64}(undef, 2, nI)
        Jcoords = Matrix{Float64}(undef, 2, nJ)
        i = 1
        do_J = true
        while !eof(f)
            line = readline(f)
            sline = split(line)
            if do_J
                Jcoords[1,i] = parse(Float64,sline[1])
                Jcoords[2,i] = parse(Float64,sline[2])
                if i == nJ
                    i = 0
                    do_J = false
                end
            else
                Icoords[1,i] = parse(Float64,sline[1])
                Icoords[2,i] = parse(Float64,sline[2])
            end
            i += 1
        end
        return nI, nJ, Icoords, Jcoords
    end    
end

In [ ]:
df = DataFrame(id=Int[], in_type=String[], nI=Int[], nJ=Int[], cv=Float64[], D=Int[], FLR=Float64[], FCR=Int[])

params0 = SSD.default_params()
data = SSD.default_data()

coords_bounds=(0,100)
r_bounds=(0.3, 0.4)
k=5
t=3
a_bounds=(80,120)

id = 1

in_type = "Own"
for (nI, nJ) in [(20,3), (50,10), (70,50)]
    for cv in [0.5, 1, 1.5]
        for D in [1, 10, 25, 50, 100]
            for FLR in [0.4, 0.6, 0.8]
                for FCR in [2,4,8,10]
                    push!(df, (id, in_type, nI, nJ, cv, D, FLR, FCR));
                    SSD.instance_gen(id, in_type, nI, nJ, coords_bounds, a_bounds, r_bounds, cv, D, k, t, FLR, FCR, params0)
                    id += 1
                end
            end
        end
    end
end

in_type = "p-med"
directory = "instances/p-median/"
                # (90,10)   (185,15)        (275,25)     (372,30)      (2338, 700)
for filename in ["SJC1.dat", "SJC2.dat", "SJC3a.dat", "SJC4a.dat"]#, "p3038_700.dat"]
    file = directory*filename
    nI, nJ, Icoords, Jcoords = extract_pmed(file)
    for cv in [0.5, 1, 1.5]
        for D in [1, 10, 25, 50, 100]
            for FLR in [0.4, 0.6, 0.8]
                for FCR in [2,4,8,10]
                    push!(df, (id, in_type, nI, nJ, cv, D, FLR, FCR));
                    SSD.instance_gen(id, in_type, nI, nJ, coords_bounds, a_bounds, r_bounds, cv, D, k, t, FLR, FCR, params0, Icoords, Jcoords)
                    id += 1
                end
            end
        end
    end
end

In [ ]:
CSV.write("instances/instances inventory.csv",df)

In [ ]:
# Create instances

df = DataFrame(id=Int[], in_type=String[], nI=Int[], nJ=Int[], cv=Float64[], D=Int[], FLR=Float64[], FCR=Int[])

params0 = SSD.default_params()
data = SSD.default_data()

coords_bounds=(0,100)
r_bounds=(0.3, 0.4)
# k=5
# t=3
a_bounds=(50,320)

id = 1

in_type = "Own"
for (nI, nJ) in [(30,5), (200,30), (500,100)] #[(20,3), (70,10), (250,10)]
    for t in [4, 12, 54]
        for k in [5, 10]
            for cv in [1]
                for D in [72500]
                    for FLR in [0.6]
                        for FCR in [8]
                            push!(df, (id, in_type, nI, nJ, cv, D, FLR, FCR));
                            SSD.instance_gen(id, in_type, nI, nJ, coords_bounds, a_bounds, r_bounds, cv, D, k, t, FLR, FCR, params0)
                            id += 1
                        end
                    end
                end
            end
        end
    end
end

In [ ]:
# Read the outputs for the relaxed tests

df_rel = DataFrame(id=Int[], size=String[], method=String[], 
    Relax_iters=Int64[], Relax_cuts=Int64[], OF=Float64[], CPU=Float64[], Memory=Float64[])
errors_rel=[]

directory = "../output_rel/"
filenames = []
for (root, dirs, files) in walkdir(directory)
    filenames = files
end

for filename in filenames
    
    file = directory*filename
    # file=directory*"ssd_1558214_10.out"
    # println("Processing $filename")
    
    open(file,"r") do f
        take_info = false
        row = []
        benders_sec = false
        method = ""
        
        while !eof(f)
            line = readline(f)
            sline_temp = split(line,":")
            if sline_temp[1]=="solving instance SSD id"
                inst_id = parse(Int64,split(sline_temp[2], ",")[1])
                method = sline_temp[end]
                push!(row, inst_id)
                push!(row, "S_M")
                push!(row, method)
            end

            if line == "RESULTS"
                take_info = true
            end
            # println("Line: $line")
            # println("Line length: $(length(line))")
            if take_info && length(line) > 0
                sline = split(line)
                data = sline[1]
                # println("$data is equal to Cap $(data=="Cap")")
                # println("Method is $method")
                if data == "LP"
                    push!(row, 0)
                    push!(row, 0)
                    text = sline[3]
                    push!(row, round(parse(Float64,text), digits=2))
                elseif data == "Rel" && sline[2]== "iters:"
                    text = replace(sline[3], "Any"=>"", "["=>"", "]"=>"", ","=>"") 
                    push!(row, parse(Int64,text))
                elseif data == "Rel" && sline[2]== "cuts:"
                    text = replace(sline[3], "Any"=>"", "["=>"", "]"=>"", ","=>"") 
                    push!(row, parse(Int64,text))
                elseif data == "Rel" && sline[2]== "lb:"
                    text = replace(sline[3], "Any"=>"", "["=>"", "]"=>"", ","=>"") 
                    push!(row, round(parse(Float64,text), digits=2))
                elseif data == "CPU_t:"
                    text = sline[2]
                    push!(row, parse(Float64,text))
                elseif data == "Memory:"
                    text =sline[2]
                    push!(row, parse(Float64,text))
                end
            end
        end
        # println(row)
        try
            push!(df_rel, row)
        catch
            push!(errors_rel, parse(Int64,split(split(filename,"_")[end],".")[begin]))
        end
    end
    # break
end

In [ ]:
# Export the data frame
sort!(df_rel, [:id, :size], rev=[false, true])
file_name = "111024_results_relaxations.csv"

CSV.write(file_name, df_rel)

In [ ]:
df_rel

In [ ]:
sort(errors_rel)

In [ ]:
sline

In [ ]:
line="y=Any[0, 0, 0, 10, 10, 10, 0, 0, 10, 0, 10, 0, 0, 0, 0, 10, 10, 0, 0, 10, 0, 10, 10, 0, 0, 0, 10, 0, 10, 0]"
sline = split(line)
sline = [replace(i,"y=Any"=>"", "["=>"", "]"=>"", ","=>"") for i in sline]
[parse(Int64,sline[i]) for i in 1:length(sline)]

In [ ]:
# Read the outputs

df = DataFrame(id=Int[], size=String[], method=String[], OF=Float64[], LC=Float64[], AL=Float64[], 
    CC=Float64[], y=Array[], nVars=Int64[], nCons=Array[], Nodes=Int64[], Feas_cuts=Int64[], Opt_cuts=Int64[], 
    Relax_iters=Array[], Relax_cuts=Array[], Relax_bounds=Array[],
    test_cap=String[], test_1cap=String[], test_1dem=String[], outer_cuts=Array[], CPU=Float64[], Memory=Float64[], Status=String[], Iter=Int64[])
# df = DataFrame(id=Int[], inst_type =String[], method=String[], OF=Float64[], LC=Float64[], AL=Float64[], 
#     CC=Float64[], y=Array[], Nodes=Int64[], Feas_cuts=Int64[], Opt_cuts=Int64[], Relax_cuts=Array[],
#     test_cap=String[], test_1cap=String[], test_1dem=String[], CPU=Float64[], Memory=Float64[])
errors=[]

directory = "../output/"
filenames = []
for (root, dirs, files) in walkdir("../output/")
    filenames = files
end

for filename in filenames
    
    file = directory*filename
    # file=directory*"ssd_1558214_10.out"
    println("Processing $filename")
    
    open(file,"r") do f
        take_info = false
        row = []
        method=""
        benders_sec = false
        rel_lb_temp = 0
        rel_ub_temp = 0
        while !eof(f)
            line = readline(f)
            sline_temp = split(line,"=")
            if sline_temp[1]== "With this agg the bounds are lb" #"With the agg Any[\"K\"] the bounds are lb"
                rel_lb_temp = parse(Float64,split(sline_temp[2])[1])
                rel_ub_temp = parse(Float64,split(sline_temp[end])[1])
            end

            if line == "RESULTS"
                take_info = true
            end
            # println("Line: $line")
            # println("Line length: $(length(line))")
            if take_info && length(line) > 0
                sline = split(line)
                data = sline[1]
                # println("$data is equal to Cap $(data=="Cap")")
                if data == "Instance:"
                    text = sline[2]
                    push!(row, parse(Int64,split(split(text,"/")[end],".")[begin]))
                    inst_type = "S_M"
                    if split(filename,"_")[2] == "large"
                        inst_type = "L"
                    end
                    push!(row, inst_type)
                elseif data == "Method:"
                    method = sline[2]
                    push!(row, method)
                elseif data == "OF:"
                    text = sline[2]
                    push!(row, parse(Float64,text))
                elseif data == "Location"
                    text = sline[3]
                    push!(row, parse(Float64,text))
                elseif data == "Allocation"
                    text = sline[3]
                    push!(row, parse(Float64,text))
                elseif data == "Congestion"
                    text = sline[3]
                    push!(row, parse(Float64,text))
                elseif string(data[1])=="y"
                    sline = [replace(i,"y=Any"=>"", "["=>"", "]"=>"", ","=>"") for i in sline]
                    push!(row, [parse(Int64,i) for i in sline])
                    benders_sec = true
                    # if method == "lazy_cuts" || method == "iter_cuts"
                    #     push!(row, 0)
                    #     push!(row, [0])
                    #     push!(row, 0)
                    #     push!(row, 0)
                    #     push!(row, 0)
                    #     push!(row, [0])
                    #     push!(row, [0])
                    #     push!(row, [0,0])
                    #     benders_sec = false
                    # end
                elseif benders_sec
                    if data == "Vars:"
                        text = sline[2]
                        push!(row, parse(Int64,text))
                    elseif data == "Cons:"
                        sline_num = [replace(sline[i],"Any"=>"", "["=>"", "]"=>"", ","=>"") for i in 2:length(sline)]
                        push!(row, [parse(Int64,sline_num[i]) for i in 1:length(sline_num)])
                        # push!(row, parse(Int64,text))
                    elseif data == "nodes:"
                        text = sline[2]
                        push!(row, parse(Int64,replace(text,"Any"=>"", "["=>"", "]"=>"", ","=>"") )) 
                    elseif data == "Feas"
                        text = sline[3]
                        push!(row, parse(Int64,text))
                    elseif data == "Opt"
                        text = sline[3]
                        push!(row, parse(Int64,text))
                    elseif data == "Rel"
                        sline_num = [replace(sline[i],"Any"=>"", "["=>"", "]"=>"", ","=>"") for i in 3:length(sline)]
                        if sline_num != [""]
                            push!(row, [parse(Int64,sline_num[i]) for i in 1:length(sline_num)])
                        else
                            push!(row, [0])
                        end
                        # if sline[2] == "iters"
                        #     push!(row, [parse(Int64,sline_num[i]) for i in 1:length(sline_num)])
                        # elseif sline[2] == "cuts"
                        #     text = sline[3]
                        #     push!(row, [parse(Int64,sline[i][end-1]) for i in 3:length(sline)])
                        # end
                        if sline[2] == "cuts:"
                            push!(row, [rel_lb_temp,rel_ub_temp])
                            benders_sec = false
                        end
                    end
                    # end
                elseif data=="Cap"  
                    text = sline[3]                    
                    push!(row, text)
                elseif data == "1_cap:"
                    text = sline[2]
                    push!(row, text)
                elseif data == "1_demand:"
                    text = sline[2]
                    push!(row, text)
                elseif data=="Outer_cuts:"
                    text = parse.(Int64,replace(sline[i],"Any"=>"", "["=>"", "]"=>"", ","=>"") for i in 2:length(sline))
                    push!(row, text)
                elseif data == "CPU_t:"
                    text = sline[2]
                    push!(row, parse(Float64,text))
                elseif data == "Memory:"
                    text =sline[2]
                    push!(row, parse(Float64,text))
                elseif data == "Status:"
                    text =sline[2]
                    push!(row, text)
                elseif data == "Iter:"
                    text =sline[2]
                    push!(row, parse(Int64,text))
                end
            end
        end
        # println(row)
        try
            push!(df, row)
        catch
            push!(errors, parse(Int64,split(split(filename,"_")[end],".")[begin]))
        end
    end
    # break
end

In [ ]:
sort!(df, [:id, :size, :method], rev=[false, true, true])
df#[end-1,:]

In [ ]:
sort(errors)

In [ ]:
# Export the data frame
sort!(df, [:id, :size, :method], rev=[false, true, true])
file_name = "151124_results2.csv"

CSV.write(file_name, df)

In [ ]:
sort!(df, [:id, :size, :method], rev=[false, true, true])
filter(row -> row.size == "S_M", df)

In [ ]:
errors

# Clustering

In [ ]:
directory = "instances/Own/"
filename= "1.txt" #"1.txt" # "25.txt" # "6.txt"#"18_3.txt"

In [ ]:
# Create default parameters
params = SSD.default_params()
# Create the data container
data = SSD.default_data()
# Read the instance
SSD.read_file(directory*filename, data)

In [ ]:
x=data.Jcoords[1,:]
y=data.Jcoords[2,:]
scatter(x,y)

In [ ]:
n_clus = max(2,Int64(floor(data.J/10)))
res_clus = kmeans(data.Jcoords, n_clus) 
@assert nclusters(res_clus) == n_clus

In [ ]:
nclusters(res_clus), n_clus, counts(res_clus)

In [ ]:
for clus in 1:nclusters(res_clus)
    fac = findall(==(clus),assignments(res_clus))
    x=data.Jcoords[1,fac]
    y=data.Jcoords[2,fac]
    if clus == 1
        scatter(x,y)
    else
        display(scatter!(x,y))
    end
end

In [ ]:
R = []
K = 1:data.k
for clus in 1:nclusters(res_clus)
    Rl=[]
    fac = findall(==(clus),assignments(res_clus))
    for f in fac
        for k in K
            push!(Rl, (f,k))
        end
    end
    push!(R, Rl)
end

In [ ]:
z=[1 2 3; 4 5 6; 7 8 9]
a=[(1,1), (2,2), (3,2)]
sum(z[j,k] for (j,k) in a)    

In [254]:
# Test toy example
using JuMP
using MathOptInterface

In [255]:
y1_ref = 4

4

In [256]:
mp = Model(optimizer_with_attributes(()->Gurobi.Optimizer(),
        # "OutputFlag" => 0,
        "Threads" => 1,
        "MIPGap" => 1e-5,
        "Presolve" => 0,
        "Cuts" => 0,
        "LogToConsole" => 0,
        ))

@variable(mp, 0<=y1, Int)
@variable(mp, 1 <= α)

@objective(mp, Min, 2*y1 + α)

@constraint(mp, y1 <= 10)

optimize!(mp)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-18
Set parameter Threads to value 1
Set parameter MIPGap to value 1e-05
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Cuts to value 0
Set parameter MIPGap to value 1e-05
Set parameter Threads to value 1


In [257]:
value(y1)

-0.0

In [258]:
m = Model(optimizer_with_attributes(()->Gurobi.Optimizer(),
        # "OutputFlag" => 0,
        "Threads" => 1,
        "MIPGap" => 1e-5,
        "Presolve" => 0,
        "Cuts" => 0,
        "LogToConsole" => 0,
        ))

@variable(m, 0 <= y2)
@variable(m, 0 <= y3)

@objective(m, Min, 4*y2+(11/5)*y3)

@constraint(m, c1_sp, y2+y3>=20-y1_ref)  # π1
@constraint(m, c2_sp,  2*y2+10*y3>=12) # π2
@constraint(m, c3_sp,  -y3>=-y1_ref) # π3
@constraint(m, c4_sp,  -y2 >= - 25) # π4
@constraint(m, c5_sp,  -y3 >= - 25) # π5

optimize!(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-18
Set parameter Threads to value 1
Set parameter MIPGap to value 1e-05
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Cuts to value 0
Set parameter MIPGap to value 1e-05
Set parameter Threads to value 1


In [259]:
# Optimal values
value(y2), value(y3), objective_value(m)

(12.0, 4.0, 56.8)

In [260]:
π1 = dual(c1_sp)
π2 = dual(c2_sp)
π3 = dual(c3_sp)
π4 = dual(c4_sp)
π5 = dual(c5_sp)

π1, π2, π3, π4, π5

(4.0, 0.0, 1.7999999999999998, 0.0, 0.0)

In [267]:
# Expression
expr = AffExpr(0)
add_to_expression!(expr,π1*(20-mp[:y1]))
add_to_expression!(expr,π2*(12))
add_to_expression!(expr,π3*(-mp[:y1]))
add_to_expression!(expr,π4*(25))
add_to_expression!(expr,π5*(25))

@constraint(mp, α>= expr)

5.8 y1 + α >= 80

In [268]:
optimize!(mp)

In [269]:
value(y1), value(α)

(10.0, 21.999999999999996)

# Solve 1 inst

In [20]:
# Create default parameters
params = SSD.default_params()
# Create the data container
data = SSD.default_data()
# Read the instance
SSD.read_file(directory*filename, data)
data.F = SSD.gen_costs(data, params)
data.Q = SSD.gen_caps(data, params)
# Initialize the status
status = SSD.init_solver_status()

ρ_h = SSD.ini_ρ_h(data, 32) ;

J=1:data.J
I=1:data.I
T=1:data.t
K=1:data.k
cv=data.cv
D=data.D
C=data.C;


instance file instances/Own/5.txt parsed successfully



In [ ]:
M= SSD.calc_big_M(data, ρ_h)

In [145]:
y=SSD.gen_y(data,[0,1,3,0,5])

5×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [153]:
findall(==(1),y[3,:])

1-element Vector{Int64}:
 3

In [147]:
for j in J
    

12345678910111213141516171819202122232425

In [130]:
data.k

5

In [30]:
directory = "instances/Own/"
filename= "1.txt" #"1.txt" # "25.txt" # "6.txt"#"18_3.txt"

"1.txt"

In [31]:
solve_method = "benders_lazyBp"  #"nlp" "iter_cuts" "lazy_cuts" "benders_lazy" "benders_iter" "bendersSH" "bendersWS" "bendersPK"

println("Starting $solve_method")

el = @CPUelapsed res = SSD.solve_ssd(directory*filename, [solve_method], 170);
# el = @CPUelapsed res = SSD.solve_ssd(directory*filename, [""])

Starting benders_lazyBp

instance file instances/Own/1.txt parsed successfully

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-18
Set parameter Threads to value 1
Set parameter MIPGap to value 1e-05
Set parameter TimeLimit to value 18001
Set parameter LazyConstraints to value 1
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Threads to value 1
Set parameter MIPGap to value 1e-05
Set parameter TimeLimit to value 18001
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Threads to value 1
Set parameter MIPGap to value 1e-05
Set parameter TimeLimit to value 18001
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Threads to value 1
Set parameter MIPGap to value 1e-05
Set parameter TimeLimit to value 18001
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter Threads to value 1
Set parameter MIPGap to value 1e-05
Set parameter TimeLimit to valu

In [33]:
println("$(split(filename,".")[1]) - $solve_method: of=$(res[1]), of_term1=$(res[2]), of_term2=$(res[3]), of_term3=$(res[4]), y=$(res[5]), CPU_t=$el, nodes=$(res[7].nIter),
        feas=$(res[7].nFeasCuts), opt=$(res[7].nOptCuts), n_vars=$(res[end-4]), n_cons=$(res[end-3]), n_nodes=$(res[end-2]), n_tan=$(res[end])")

1 - benders_lazyBp: of=2650.51, of_term1=677.58, of_term2=1456.42, of_term3=516.51, y=Any[5, 5, 5, 0, 0], CPU_t=3.422, nodes=144,
        feas=0, opt=540, n_vars=29, n_cons=Any[67, 67], n_nodes=1295, n_tan=[170, 313]


In [21]:
data.I, data.J, data.k, data.t

(30, 5, 5, 54)

In [ ]:
1 - lazy_cuts: of=2650.51, of_term1=677.58, of_term2=1456.44, of_term3=516.49, y=Any[5, 5, 5, 0, 0], CPU_t=0.547, nodes=8,
        feas=64, opt=0, n_vars=865, n_cons=Any[5390], n_nodes=Any[1], n_tan=[170, 177]
1 - benders_lazyBp_v2: of=2650.56, of_term1=677.58, of_term2=1456.08, of_term3=516.89, y=Any[5, 5, 5, 0, 0], CPU_t=9.375, nodes=804,
        feas=3076, opt=1122, n_vars=269, n_cons=Any[4071, 4071], n_nodes=414, n_tan=[170, 563]

In [ ]:
5 - lazy_cuts: of=15528.71, of_term1=1882.36, of_term2=12410.45, of_term3=1235.9, y=Any[5, 5, 5, 5, 5], CPU_t=6.921, nodes=6,
        feas=405, opt=0, n_vars=11365, n_cons=Any[72390], n_nodes=Any[1], n_tan=[170, 174]
5 - iter_cuts: of=15528.72, of_term1=1882.36, of_term2=12410.44, of_term3=1235.91, y=Any[5, 5, 5, 5, 5], CPU_t=8.031, nodes=1,
        feas=0, opt=0, n_vars=11365, n_cons=Any[72390], n_nodes=Any[1], n_tan=[170, 171]
5 - benders_lazyBp: of=15528.75, of_term1=1882.36, of_term2=12410.48, of_term3=1235.9, y=Any[5, 5, 5, 5, 5], CPU_t=75.25, nodes=148,
        feas=4362, opt=1429, n_vars=3319, n_cons=Any[54271, 54271], n_nodes=283, n_tan=[170, 237]
5 - benders_lazyBp: of=15528.71, of_term1=1882.36, of_term2=12410.46, of_term3=1235.89, y=Any[5, 5, 5, 5, 5], CPU_t=42.313, nodes=120,
        feas=0, opt=5832, n_vars=79, n_cons=Any[217, 217], n_nodes=1541, n_tan=[170, 279]

In [143]:
1754.64+12410.53+1475

15640.17

In [51]:
sum(data.a,dims=1)

1×54 Matrix{Float64}:
 7618.0  7435.0  7596.0  7691.0  7560.0  …  7528.0  7720.0  7498.0  7407.0

In [64]:
data.Q[4,1]

1620.21

In [118]:
y=res[5]
x=res[6];

In [139]:
# Calc the free cap
x=res[6]
j = 2
max_dem_j = 0
for t in 1:data.t
    dem_j = sum(data.a[:,t]'*x[:,:,t],dims=1)
    if dem_j[j] > max_dem_j
        max_dem_j = dem_j[j]
    end
    # println("$t $()") #, assert $(sum(data.a[:,t])) == $(sum(sum(data.a[:,t]'*x[:,:,t],dims=1)))")
end
max_dem_j

1535.0000000000052

In [140]:
data.Q[1,5], data.Q[2,5], data.Q[3,5], data.Q[4,2], data.Q[5,5]
# 2260.9,  1535,  2083,  1302, 1593

(4860.62, 4860.62, 4860.62, 2430.31, 4860.62)

In [135]:
dot(data.F, SSD.gen_y(data,res[5]))

1754.6399999999999

In [136]:
dot(data.C, x)

12410.484834940262

In [129]:
function calc_con(data, y, x)
    D = data.D
    T = 1:data.t
    J = 1:data.J
    I = 1:data.I
    λ = data.a
    cv = data.cv
    Q = data.Q
    q = [Q[j,y[j]] for j in J]
    # q[4]=Q[j,2]
    Dt = [D/sum(λ[:,t]) for t in T]
    Cong = []
    for t in T
        dem_j = sum(data.a[:,t]'*x[:,:,t],dims=1)
        R_term = sum((1+cv^2)*(dem_j[j]/(q[j]-dem_j[j])) for j in J)
        ρ_term = sum((1-cv^2)*(dem_j[j]/(q[j])) for j in J)
        cong_t = 0.5*Dt[t]*(R_term+ρ_term)
        push!(Cong, cong_t)
    end
    return Cong
end

calc_con (generic function with 1 method)

In [137]:
Cong_t = calc_con(data, res[5], res[6])

54-element Vector{Any}:
 27.415774928511425
 26.178415243909722
 27.125827920706275
 27.28214833044362
 27.273095502834003
 27.51262429687213
 27.342061621076454
 27.39681089111671
 27.34766916507656
 27.25008072512758
 27.275235484490317
 27.36036101195968
 26.8848571464156
  ⋮
 29.61999672416289
 27.577281768939205
 27.187282027214145
 27.483881775438846
 27.00468005761698
 27.735911459353392
 27.287164277169023
 27.63037235664449
 27.29765495740424
 29.376299034036688
 26.96995058536641
 27.413422676730566

In [138]:
sum(Cong_t)

1475.1439455799723

In [105]:
sum(Cong_t)

1672.2256920629902

In [102]:
sum(Cong_t)

1235.9033821821547

In [ ]:
1800.52

## Is the solution correct?

In [ ]:
data0.I, data0.J, data0.k, data0.t

In [ ]:
# Allocation decisions
x = res[7]
# Location decisions
y = SSD.gen_y(data0, res[6]);

### Is feasible?

In [ ]:
# See if the demand can be allocated with the given capacity

In [ ]:
# Facility capacity constraint
# For each time period, see the allocation of customers
for t in 1:data0.t
    sum(round.(x[:,:,t]'data0.a[:,t], digits=3) .> sum(y.*data0.Q,dims=2)) > 0 ? println("Bad for t=$t") : println("Ok for t=$t")
end

In [ ]:
# All the demand is assigned
for t in 1:data0.t
    for i in 1:data0.I
        sum(x[i,j,t] for j in 1:data0.J) == 1 ? print("") : println("Bad for node $i in t=$t") #println("Ok for node $i in t=$t")
    end
end

In [ ]:
# Only one cap level selected
for j in 1:data0.J
    sum(y[j,k] for k in 1:data0.k) == 1 ? print("") : sum(y[j,k] for k in 1:data0.k) == 0 ? print("") : println("Bad for facility $j")
end

# Plot de bounds

In [ ]:
#of (1), Fterm, Allocterm, Congterm, y (5), x, status, lb, ub, relax_iters (10), cuts_iters, n_vars, n_cons, n_nodes, tests_feas (15)
ordered_lb = sort(res[8])
ordered_ub = sort(res[9])

of_val = 18049.4039

# LB
plot(0:res[7].nIter, collect(values(ordered_lb)), label="lb",
    size=(1000,400), markershape=:xcross, legend=:topleft)

# UB
plot!(0:res[7].nIter, collect(values(ordered_ub)), label="ub",
    size=(1000,400), markershape=:xcross, color=:red)

# LP relaxation optimal
# plot!(1:status.nIter, zeros(status.nIter).+of_val_rel, label="lp relax optimal", color=:orange, linestyle = :dash)
# Optimal
plot!(1:res[7].nIter, zeros(res[7].nIter).+of_val, label="optimal", color=:red, linestyle = :dash)

ylims!(0, of_val*1.5)

In [ ]:
# Instance Own/18_3 with bendersIter
ordered_lb = sort(res[12])
ordered_ub = sort(res[13])

# of_val = 7555.2078
# of_val_rel = 5951.6929
# gaps = [10e-3, 5e-2, 10e-2]
# refs = [of_val*(1-i) for i in gaps]
plot(1:status.nIter, collect(values(ordered_lb))[2:end])#, label="lb")#,
    # size=(1000,400), markershape=:xcross, legend=:topleft)

# LP relaxation optimal
# plot!(1:status.nIter, zeros(status.nIter).+of_val_rel, label="lp relax optimal", color=:orange, linestyle = :dash)
# Optimal
# plot!(1:status.nIter, zeros(status.nIter).+of_val, label="optimal", color=:red, linestyle = :dash)
# Gaps
# for i in 1:length(gaps)
#     gap = gaps[i]
#     plot!(1:status.nIter, zeros(status.nIter).+refs[i], label=gap, linestyle = :dash)
# end
# plot!([res[end-1]+1,res[end-1]+1],[collect(values(ordered_lb))[1],of_val], label="start branching", color=:black)

# title!("I = 250, J = 10 with $solve_method - Papadakos (3h - time lim)")

# xlims!(0, 100)

In [ ]:
ordered_lb = sort(res[end-1])
ordered_ub = sort(res[end]);
# Instance Own/18_3 with benders_cb
of_val = 7555.2078
of_val_rel = 5070.6749
gaps = [10e-3, 5e-2, 10e-2, round(abs(of_val-last(collect(values(ordered_ub)))/of_val),digits=2)]
refs = [of_val*(1-i) for i in gaps]
plot(1:status.nIter, collect(values(ordered_ub)), label="ub",
    size=(1000,400), markershape=:xcross, legend=:topleft)

# LP relaxation optimal
plot!(1:status.nIter, zeros(status.nIter).+of_val_rel, label="lp relax optimal", color=:orange, linestyle = :dash)
# Optimal
plot!(1:status.nIter, zeros(status.nIter).+of_val, label="optimal", color=:red, linestyle = :dash)
# Gaps
# for i in 1:length(gaps)
#     gap = gaps[i]
#     plot!(1:status.nIter, zeros(status.nIter).+refs[i], label=gap, linestyle = :dash)
# end
# plot!([res[end-1]+1,res[end-1]+1],[collect(values(ordered_lb))[1],of_val], label="start branching", color=:black)

title!("I = 250, J = 10 with $solve_method - Papadakos (163 s)")

xlims!(0, 100)

In [ ]:
# Instance Own/6
of_val = 1914.4834
gaps = [10e-3, 5e-2, 10e-2, round(abs(of_val-ordered_lb[res[end-1]])/of_val,digits=2)]
refs = [of_val*(1-i) for i in gaps]
plot(1:status.nIter, collect(values(ordered_lb)), label="lb",
    size=(1000,400), markershape=:xcross)
plot!(1:status.nIter, collect(values(ordered_ub)), label="ub", markershape=:xcross)
# ylims!(1500, 4000)
# Optimal
plot!(1:status.nIter, zeros(status.nIter).+of_val, label="optimal", color=:red, linestyle = :dash, lw=1)
# Gaps
for i in 1:length(gaps)
    gap = gaps[i]
    plot!(1:status.nIter, zeros(status.nIter).+refs[i], label=gap, linestyle = :dash, lw=1)
    annotate!(status.nIter, refs[i], text("$gap", :left, 12, :black))
end
plot!([res[end-1]+1,res[end-1]+1],[collect(values(ordered_lb))[1],of_val], label="start branching", color=:black)

title!("I = 20, J = 3")